In [2]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt 
import torch 

import warnings
warnings.filterwarnings("ignore")

In [13]:
idxc = pd.read_csv('../task3/clean_data/clean_idxc.csv')
y_pred = pd.read_csv('../task3/clean_data/clean_pred.csv')
y_true = pd.read_csv('../task3/clean_data/clean_true.csv')
rf = pd.read_csv('../task3/clean_data/clean_ortho_rf.csv')
factor_ret = pd.read_csv('../task3/clean_data/clean_factor_ret.csv')

In [14]:
def fillzeros(y_true):
    code_idx = np.sort(np.array(list(set(y_true['stock_code']))))
    date_idx = np.sort(np.array(list(set(y_true['trade_date']))))
    code_fill = []
    for code in code_idx:
        x = y_true[y_true['stock_code']==code]
        if len(x)<339:
            date_fill = list(set(date_idx)-set(x['trade_date']))
            df = pd.DataFrame({'trade_date':date_fill,'stock_code':[code]*len(date_fill),'stock_return':[0.0]*len(date_fill)})
            y_true = pd.concat([y_true,df],axis=0,ignore_index=True)
            
    y_true = y_true.sort_values(['trade_date','stock_code'])
    y_true.index = range(len(y_true))
    return y_true

In [15]:
# 把stock_return调整为前后两天的价格比，并加入现金部分，记为stock_code=-1
def add_cash(y_true):
    date_idx = np.sort(np.array(list(set(y_true['trade_date']))))
    y_true['stock_return']+=1
    y_true = y_true.rename(columns={'stock_return':'y_t'})
    for date in date_idx:
        cash = pd.DataFrame({'trade_date':[date],'stock_code':[-1],'y_t':[1]})
        y_true = pd.concat([y_true,cash],axis=0,ignore_index=True)
    y_true = y_true.sort_values(['trade_date','stock_code'])
    y_true.index = range(len(y_true))
    return y_true

In [16]:
# 对持仓权重表进行结构化（对不持仓的股票的weight补零，并添加现金部分）
def struct(df_weight,y_true):
    date_idx = np.sort(np.array(list(set(df_weight['trade_date']))))
    new_df_weight = pd.DataFrame()
    for date in date_idx:
        temp = df_weight[df_weight['trade_date']==date].set_index('stock_code')
        temp2 = y_true[y_true['trade_date']==date].set_index('stock_code')
        # 对不持仓的股票的weight补零
        temp3 = pd.concat([temp,temp2],axis=1,join='outer').iloc[:,:2].fillna(0)
        temp3['trade_date'] = date
        temp3 = temp3.reset_index()
        # 添加现金项
        temp3.loc[0,'weight'] = 1-temp3[temp3['stock_code']!=-1]['weight'].sum()
        new_df_weight = pd.concat([new_df_weight,temp3],axis=0)
    return new_df_weight

In [17]:
y_true_fillzeros = fillzeros(y_true)
y_true_with_cash = add_cash(y_true_fillzeros)

In [8]:
idxc_struct = struct(idxc,y_true_with_cash)

In [12]:
y_true_with_cash.to_csv('../task3/clean_data/clean_y_true_with_cash.csv',index=False)
idxc_struct.to_csv('../task3/clean_data/clean_idxc_struct.csv',index=False)